In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.0.0'

In [4]:
#动态图
tf.executing_eagerly()

True

In [14]:
x = [[2.]]
m = tf.matmul(x, x)
print('x matmul x: {}'.format(m))

x matmul x: [[4.]]


In [23]:
#tensor obj
a = tf.constant([[1,2],
                 [3,4]])
a

<tf.Tensor: id=29, shape=(2, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4]])>

In [25]:
#broadcasting
b = tf.add(a,1)
b

<tf.Tensor: id=33, shape=(2, 2), dtype=int32, numpy=
array([[2, 3],
       [4, 5]])>

In [27]:
a*b

<tf.Tensor: id=35, shape=(2, 2), dtype=int32, numpy=
array([[ 2,  6],
       [12, 20]])>

In [28]:
tf.matmul(a,b)

<tf.Tensor: id=36, shape=(2, 2), dtype=int32, numpy=
array([[10, 13],
       [22, 29]])>

In [39]:
import numpy as np

In [44]:
c = np.multiply(a,b)
c

array([[ 2,  6],
       [12, 20]])

In [49]:
a.numpy()

array([[1, 2],
       [3, 4]])

In [55]:
#tape计算梯度
w = tf.Variable([[1.0]])
with tf.GradientTape() as tape:
    loss = w * w
grad = tape.gradient(loss, w)
print(grad)

tf.Tensor([[2.]], shape=(1, 1), dtype=float32)


train a model

In [140]:
#data prepare
(x_train,y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [141]:
print(x_train.shape)
print(y_train.shape)

(60000, 28, 28)
(60000,)


In [142]:
x_train = x_train[:10000,:,:]
y_train = y_train[:10000]
x_test = x_test[:1000,:,:]
y_test = y_test[:1000]

In [143]:
x_train[...,tf.newaxis].shape

(10000, 28, 28, 1)

In [144]:
y_train[0]

5

In [145]:
x_train = tf.cast(x_train[...,tf.newaxis]/255,tf.float32)
x_test = tf.cast(x_test[...,tf.newaxis]/255,tf.float32)

In [189]:
x_train[0]

<tf.Tensor: id=9560, shape=(28, 28, 1), dtype=float32, numpy=
array([[[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0. 

In [147]:
tf.keras.utils.to_categorical(y_train,10)[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [148]:
y_train = tf.keras.utils.to_categorical(y_train,10)
y_test = tf.keras.utils.to_categorical(y_test,10)

In [149]:
#build model with sequential
mnist_model = tf.keras.Sequential([
                tf.keras.layers.Conv2D(32,[3,3],activation = 'relu',input_shape = (28,28,1)),
                tf.keras.layers.Conv2D(64,[3,3],activation = 'relu'),
                tf.keras.layers.MaxPooling2D(pool_size = (2,2)),  #stride value default = pool_size
                tf.keras.layers.Dropout(0.25),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(128, activation = 'relu'),
                tf.keras.layers.Dropout(0.5),
                tf.keras.layers.Dense(10, activation = 'softmax')
                ])

In [150]:
mnist_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)              

In [96]:
#build the model using Model
inputs = tf.keras.Input(shape = (28,28,1),name = 'digits')
conv_1 = tf.keras.layers.Conv2D(16,[3,3],activation = 'relu')(inputs)
conv_2 = tf.keras.layers.Conv2D(16,[3,3],activation = 'relu')(conv_1)
ave_pool = tf.keras.layers.GlobalAveragePooling2D()(conv_2) #全局池化层省掉Dense这一步
outputs = tf.keras.layers.Dense(10)(ave_pool)
mnist_model_2 = tf.keras.Model(inputs = inputs, outputs = outputs)

In [151]:
mnist_model_2.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 24, 24, 16)        2320      
_________________________________________________________________
global_average_pooling2d_4 ( (None, 16)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                170       
Total params: 2,650
Trainable params: 2,650
Non-trainable params: 0
_________________________________________________________________


# two train method

## keras fit method

In [169]:
mnist_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
                    shuffle=True,
                   loss = tf.keras.losses.categorical_crossentropy,
                   metrics = ["accuracy",'mse'])

In [165]:
mnist_model_2.compile(optimizer = tf.keras.optimizers.Adam(),
                   loss = tf.keras.losses.categorical_crossentropy,
                   metric = ['accuracy','mse'])

In [172]:
mnist_model.fit(x_train,y_train,batch_size = 128,epochs = 3,validation_split = 0.1)

Train on 9000 samples, validate on 1000 samples
Epoch 1/3
9000/9000 [==============================] - 20s 2ms/sample - loss: 1.1717 - accuracy: 0.6554 - mse: 0.0524 - val_loss: 0.8548 - val_accuracy: 0.8220 - val_mse: 0.0373
Epoch 2/3
9000/9000 [==============================] - 18s 2ms/sample - loss: 1.0828 - accuracy: 0.6757 - mse: 0.0489 - val_loss: 0.7693 - val_accuracy: 0.8270 - val_mse: 0.0335
Epoch 3/3
9000/9000 [==============================] - 18s 2ms/sample - loss: 1.0120 - accuracy: 0.6967 - mse: 0.0460 - val_loss: 0.7014 - val_accuracy: 0.8400 - val_mse: 0.0305


In [173]:
mnist_model.evaluate(x_test,y_test)

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.8438546419143677, 0.789, 0.037722178]

In [181]:
x_test[0]

<tf.Tensor: id=9235, shape=(28, 28, 1), dtype=float32, numpy=
array([[[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0. 

In [185]:
mnist_model.predict(x_test)

array([[1.4845999e-02, 6.4246934e-03, 1.4458366e-02, ..., 8.5051697e-01,
        1.0039862e-02, 3.8142174e-02],
       [6.2811822e-02, 9.7738482e-02, 1.7156509e-01, ..., 6.0560158e-04,
        1.1006483e-01, 2.3202267e-03],
       [1.4551338e-02, 6.4437050e-01, 7.2685964e-02, ..., 2.4719600e-02,
        7.3551826e-02, 2.6295578e-02],
       ...,
       [9.8139131e-01, 3.6209771e-05, 9.4191535e-05, ..., 2.0227587e-04,
        5.1459140e-04, 4.4979166e-05],
       [4.1013360e-02, 2.9962519e-03, 1.9896809e-02, ..., 2.6433304e-01,
        6.0616255e-02, 2.2174025e-01],
       [1.7812265e-02, 1.1671551e-02, 6.7204557e-02, ..., 1.7805900e-01,
        2.5020476e-02, 3.6080196e-01]], dtype=float32)

## use TF2.0 train method

In [206]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train[:10000,:,:]
y_train = y_train[:10000]
x_test = x_test[:1000,:,:]
y_test = y_test[:1000]

In [207]:
dataset = tf.data.Dataset.from_tensor_slices(
                                            (tf.cast(x_train[...,tf.newaxis]/255,tf.float32),
                                            tf.cast(y_train,tf.int64)))
dataset = dataset.shuffle(1000).batch(32)

In [223]:
dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [208]:
optimizer = tf.keras.optimizers.SGD(learning_rate = 1e-3)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
loss_history = []

In [213]:
it = iter(dataset)
print(next(it)[1])

tf.Tensor([0 6 1 0 5 5 4 4 4 1 1 6 8 8 2 1 4 9 7 6 1 1 0 5 7 5 9 6 3 1 6 1], shape=(32,), dtype=int64)


In [216]:
for epoch in range(2):
    for (batch,(images,labels)) in enumerate(dataset):
        with tf.GradientTape() as tape:
            logits = mnist_model(images, training = True)
            loss_value = loss(labels, logits)
        grads = tape.gradient(loss_value,mnist_model.trainable_variables)
        optimizer.apply_gradients(zip(grads,mnist_model.trainable_variables))
        loss_history.append(loss_value)
    print('{} epoch is finished'.format(epoch))
    print(loss_value)

0 epoch is finished
tf.Tensor(1.9187876, shape=(), dtype=float32)
1 epoch is finished
tf.Tensor(1.5933524, shape=(), dtype=float32)
